# Querying with RedisVL

In this notebook, we will explore more complex queries that can be performed with `RedisVL`

Before running this notebook, be sure to:
1. Have installed `RedisVL` and have that environment active for this notebook.
2. Have a running Redis instance with RediSearch > 2.4 running.

## Import Required Libraries and Setup

In [1]:
// Load Maven dependencies
%maven redis.clients:jedis:5.2.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.fasterxml.jackson.core:jackson-databind:2.18.0
%maven com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.18.0
%maven net.razorvine:pickle:1.4
%maven de.vandermeer:asciitable:0.3.2

// Import RedisVL classes
import com.redis.vl.index.SearchIndex;
import com.redis.vl.schema.IndexSchema;
import com.redis.vl.query.VectorQuery;
import com.redis.vl.query.Filter;

// Import Redis client
import redis.clients.jedis.UnifiedJedis;
import redis.clients.jedis.HostAndPort;
import redis.clients.jedis.search.SearchResult;

// Import pickle library
import net.razorvine.pickle.Unpickler;

// Import ASCII Table
import de.vandermeer.asciitable.AsciiTable;
import de.vandermeer.asciitable.CWC_LongestLine;

// Import Java standard libraries
import java.util.*;
import java.nio.*;
import java.io.*;

System.out.println("Libraries imported successfully!");

Libraries imported successfully!


## Load Sample Data from Pickle File

Load the hybrid_example_data.pkl file that contains our sample dataset:

In [2]:
// Load data from pickle file and convert vector byte arrays to float arrays
Unpickler unpickler = new Unpickler();
List<Map<String, Object>> data;

try {
    FileInputStream fileInputStream = new FileInputStream("resources/hybrid_example_data.pkl");
    data = (List<Map<String, Object>>) unpickler.load(fileInputStream);
    fileInputStream.close();

    System.out.println("Sample data loaded from pickle file with " + data.size() + " users");

    // Convert byte array embeddings to float arrays
    for (Map<String, Object> user : data) {
        Object embedding = user.get("user_embedding");

        if (embedding instanceof byte[]) {
            byte[] embBytes = (byte[]) embedding;
            // Convert to float array - 3 floats * 4 bytes each = 12 bytes
            ByteBuffer buffer = ByteBuffer.wrap(embBytes).order(ByteOrder.LITTLE_ENDIAN);
            float[] floats = new float[3];
            for (int i = 0; i < 3; i++) {
                floats[i] = buffer.getFloat();
            }
            user.put("user_embedding", floats); // Replace with float array
        }
    }

    // Create ASCII table for displaying loaded data
    AsciiTable table = new AsciiTable();
    table.addRule();
    table.addRow("User", "Age", "Job", "Credit Score", "Office Location", "User Embedding", "Last Updated");
    table.addRule();

    data.forEach(user -> {
        float[] vec = (float[]) user.get("user_embedding");
        String vectorStr = String.format("[%.1f, %.1f, %.1f]", vec[0], vec[1], vec[2]);
        table.addRow(
            user.get("user"),
            user.get("age"),
            user.get("job"),
            user.get("credit_score"),
            user.get("office_location"),
            vectorStr,
            user.get("last_updated")
        );
    });
    table.addRule();

    // Configure column widths
    CWC_LongestLine cwc = new CWC_LongestLine();
    table.getRenderer().setCWC(cwc);

    System.out.println("\nLoaded data:");
    System.out.println(table.render());

} catch (Exception e) {
    System.err.println("Error loading pickle file: " + e.getMessage());
    e.printStackTrace();
    throw e;
}

Sample data loaded from pickle file with 7 users

Loaded data:
┌───────┬───┬─────────────┬────────────┬─────────────────┬───────────────┬────────────┐
│User   │Age│Job          │Credit Score│Office Location  │User Embedding │Last Updated│
├───────┼───┼─────────────┼────────────┼─────────────────┼───────────────┼────────────┤
│john   │18 │engineer     │high        │-122.4194,37.7749│[0.1, 0.1, 0.5]│1741627789  │
│derrick│14 │doctor       │low         │-122.4194,37.7749│[0.1, 0.1, 0.5]│1741627789  │
│nancy  │94 │doctor       │high        │-122.4194,37.7749│[0.7, 0.1, 0.5]│1710696589  │
│tyler  │100│engineer     │high        │-122.0839,37.3861│[0.1, 0.4, 0.5]│1742232589  │
│tim    │12 │dermatologist│high        │-122.0839,37.3861│[0.4, 0.4, 0.5]│1739644189  │
│taimur │15 │CEO          │low         │-122.0839,37.3861│[0.6, 0.1, 0.5]│1742232589  │
│joe    │35 │dentist      │medium      │-122.0839,37.3861│[0.9, 0.9, 0.1]│1742232589  │
└───────┴───┴─────────────┴────────────┴─────────────────

In [3]:
// Helper method to print query results in table format
public static void resultPrint(List<Map<String, Object>> results) {
    if (results == null || results.isEmpty()) {
        System.out.println("No results to display");
        return;
    }

    // Fields to exclude from display (keep vector_distance if present)
    Set<String> toRemove = Set.of("id", "payload", "score", "user_embedding", "__user_embedding_score");

    // Get all unique keys from results, excluding the ones we want to remove
    Set<String> allKeys = new LinkedHashSet<>();
    for (Map<String, Object> result : results) {
        for (String key : result.keySet()) {
            if (!toRemove.contains(key)) {
                allKeys.add(key);
            }
        }
    }

    // Create ASCII table
    AsciiTable table = new AsciiTable();
    table.addRule();

    // Add header row
    table.addRow(allKeys.toArray());
    table.addRule();

    // Add data rows
    for (Map<String, Object> result : results) {
        List<Object> row = new ArrayList<>();
        for (String key : allKeys) {
            Object value = result.get(key);
            row.add(value != null ? value.toString() : "");
        }
        table.addRow(row.toArray());
    }
    table.addRule();

    // Configure column widths
    CWC_LongestLine cwc = new CWC_LongestLine();
    table.getRenderer().setCWC(cwc);

    // Print the table
    System.out.println(table.render());
}

## Define Schema and Create Index

Create a more complex schema that includes geographic and timestamp fields:

In [4]:
Map<String,Object> schemaMap = Map.<String, Object>of(
  // Index configuration
  "index", Map.of(
    "name", "user_queries",
    "prefix", "user_queries_docs",
    "storage_type", "hash"
  ),
  // Field definitions
  "fields", List.of(
    Map.of("name", "user", "type", "tag"),
    Map.of("name", "credit_score", "type", "tag"),
    Map.of("name", "job", "type", "text"),
    Map.of("name", "age", "type", "numeric"),
    Map.of("name", "last_updated", "type", "numeric"),
    Map.of("name", "office_location", "type", "geo"),
    // Vector field
    Map.of(
      "name", "user_embedding",
      "type", "vector",
      "attrs", Map.of(
        "dims", 3,
        "distance_metric", "cosine",
        "algorithm", "flat",
        "datatype", "float32"
      )
    )
  )
);

int size = ((List<?>) schemaMap.get("fields")).size();
System.out.println("Schema created with " + size + " fields");

Schema created with 7 fields


In [5]:
// Connect to Redis and create index
UnifiedJedis jedis = new UnifiedJedis(new HostAndPort("redis-stack", 6379));

// Create SearchIndex
SearchIndex index = SearchIndex.fromDict(schemaMap, jedis);

// Create the index (overwrite if exists)
index.create(true);

System.out.println("Index created: " + index.getName());
System.out.println("Index exists: " + index.exists());

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Index created: user_queries
Index exists: true


In [6]:
// Load data to Redis
List<String> keys = index.load(data, "user");

System.out.println("Data loaded successfully!");
System.out.println("Document count: " + index.getDocumentCount());

// Show loaded keys
keys.forEach(key -> System.out.println("  " + key));

Data loaded successfully!
Document count: 7
  user_queries_docs:john
  user_queries_docs:derrick
  user_queries_docs:nancy
  user_queries_docs:tyler
  user_queries_docs:tim
  user_queries_docs:taimur
  user_queries_docs:joe


## Hybrid Queries

Hybrid queries are queries that combine multiple types of filters. For example, you may want to search for a user that is a certain age, has a certain job, and is within a certain distance of a location. This is a hybrid query that combines numeric, tag, and geographic filters.

### Tag Filters

Tag filters are filters that are applied to tag fields. These are fields that are not tokenized and are used to store a single categorical value.

In [7]:
// Test tag filter query
Filter tagFilter = Filter.tag("credit_score", "high");

VectorQuery vectorQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(tagFilter.build())
    .build();

List<Map<String, Object>> results = index.query(vectorQuery);
resultPrint(results);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬─────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼─────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john │18 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy│94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler│100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim  │12 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴─────┴───┘


In [8]:
// Tag negation (credit_score != high)
Filter tagNotFilter = Filter.tagNot("credit_score", "high");

VectorQuery tagNotQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(tagNotFilter.build())
    .build();

List<Map<String, Object>> tagNotResults = index.query(tagNotQuery);
resultPrint(tagNotResults);

┌────────────┬────────────┬───────────────┬─────────────────┬───────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job    │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼───────┼───────┼───┤
│low         │1741627789  │0              │-122.4194,37.7749│doctor │derrick│14 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO    │taimur │15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist│joe    │35 │
└────────────┴────────────┴───────────────┴─────────────────┴───────┴───────┴───┘


In [9]:
// Multiple tag values
Filter multiTagFilter = Filter.tag("credit_score", "high", "medium");

VectorQuery multiTagQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(multiTagFilter.build())
    .build();

List<Map<String, Object>> multiResults = index.query(multiTagQuery);
resultPrint(multiResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬─────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼─────┼───┤
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist      │joe  │35 │
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john │18 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy│94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler│100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim  │12 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴─────┴───┘


What about scenarios where you might want to dynamically generate a list of tags? Have no fear. RedisVL allows you to do this gracefully without having to check for the **empty case**. The **empty case** is when you attempt to run a Tag filter on a field with no defined values to match:

`FilterQuery.tag("credit_score")` with empty values

An empty filter like this will yield a `*` Redis query filter which implies the base case -- there is no filter here to use.

### Numeric Filters

Numeric filters are filters that are applied to numeric fields and can be used to isolate a range of values for a given field.

In [10]:
// Age range filter
Filter ageFilter = Filter.numeric("age").between(15, 35);

VectorQuery ageQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "age", "job", "credit_score", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(ageFilter.build())
    .build();

List<Map<String, Object>> ageResults = index.query(ageQuery);
resultPrint(ageResults);

┌────────────┬────────────┬───────────────┬─────────────────┬────────┬──────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job     │user  │age│
├────────────┼────────────┼───────────────┼─────────────────┼────────┼──────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer│john  │18 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO     │taimur│15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist │joe   │35 │
└────────────┴────────────┴───────────────┴─────────────────┴────────┴──────┴───┘


In [11]:
// Exact age filter
Filter exactAgeFilter = Filter.numeric("age").eq(14);

VectorQuery exactAgeQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "age", "job", "credit_score", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(exactAgeFilter.build())
    .build();

List<Map<String, Object>> exactResults = index.query(exactAgeQuery);
resultPrint(exactResults);

┌────────────┬────────────┬───────────────┬─────────────────┬──────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job   │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼──────┼───────┼───┤
│low         │1741627789  │0              │-122.4194,37.7749│doctor│derrick│14 │
└────────────┴────────────┴───────────────┴─────────────────┴──────┴───────┴───┘


### Timestamp Filters

In Redis all times are stored as an epoch time numeric however, this class allows you to filter with Java datetime objects for ease of use.

In [12]:
// Import Java time classes
import java.time.LocalDateTime;
import java.time.Instant;
import java.time.ZoneOffset;

// Create a datetime for comparison (matching Python: datetime(2025, 3, 16, 13, 45, 39, 132589))
// Note: Java LocalDateTime doesn't support microseconds, so we use nanoseconds
LocalDateTime dt = LocalDateTime.of(2025, 3, 16, 13, 45, 39, 132589000);
double epochWithMicros = dt.toEpochSecond(ZoneOffset.UTC) + (132589.0 / 1000000.0);
System.out.println("Epoch comparison: " + epochWithMicros);

// Filter for timestamps after the given datetime
Filter timestampAfterFilter = Filter.timestamp("last_updated").gt(dt);

VectorQuery timestampAfterQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "last_updated", "job", "credit_score", "age", "office_location", "vector_distance")
    .numResults(10)
    .withPreFilter(timestampAfterFilter.build())
    .build();

List<Map<String, Object>> afterResults = index.query(timestampAfterQuery);
resultPrint(afterResults);

Epoch comparison: 1.742132739132589E9
┌────────────┬────────────┬───────────────┬─────────────────┬────────┬──────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job     │user  │age│
├────────────┼────────────┼───────────────┼─────────────────┼────────┼──────┼───┤
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer│tyler │100│
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO     │taimur│15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist │joe   │35 │
└────────────┴────────────┴───────────────┴─────────────────┴────────┴──────┴───┘


In [13]:
// Filter for timestamps before the given datetime
Filter timestampBeforeFilter = Filter.timestamp("last_updated").before(dt);

VectorQuery timestampBeforeQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "last_updated", "job", "credit_score", "age", "office_location", "vector_distance")
    .numResults(10)
    .withPreFilter(timestampBeforeFilter.build())
    .build();

List<Map<String, Object>> beforeResults = index.query(timestampBeforeQuery);
System.out.println("\nTimestamps before " + dt + ":");
resultPrint(beforeResults);


Timestamps before 2025-03-16T13:45:39.132589:
┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john   │18 │
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


In [14]:
// Filter for timestamps less than the given datetime (matching Python's lt)
System.out.println("\nEpoch comparison: " + epochWithMicros);

Filter timestampBeforeFilter = Filter.timestamp("last_updated").lt(dt);

VectorQuery timestampBeforeQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "last_updated", "job", "credit_score", "age", "office_location", "vector_distance")
    .numResults(10)
    .withPreFilter(timestampBeforeFilter.build())
    .build();

List<Map<String, Object>> beforeResults = index.query(timestampBeforeQuery);
resultPrint(beforeResults);


Epoch comparison: 1.742132739132589E9
┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john   │18 │
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


### Text Filters

Text filters are applied to text fields and support various matching strategies including exact matches, wildcards, and fuzzy matching.

In [15]:
// Exact text match filter
Filter textFilter = Filter.text("job", "doctor");

VectorQuery textQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "job", "credit_score", "age", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(textFilter.build())
    .build();

List<Map<String, Object>> textResults = index.query(textQuery);
resultPrint(textResults);

┌────────────┬────────────┬───────────────┬─────────────────┬──────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job   │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼──────┼───────┼───┤
│low         │1741627789  │0              │-122.4194,37.7749│doctor│derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor│nancy  │94 │
└────────────┴────────────┴───────────────┴─────────────────┴──────┴───────┴───┘


In [16]:
// Fuzzy text filter
Filter fuzzyFilter = Filter.fuzzy("job", "dctr");

VectorQuery fuzzyQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "job", "credit_score", "age", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(fuzzyFilter.build())
    .build();

List<Map<String, Object>> fuzzyResults = index.query(fuzzyQuery);
resultPrint(fuzzyResults);

No results to display


### Geographic Filters

Geographic filters enable location-based queries using latitude/longitude coordinates and radius searches.

In [17]:
// Wildcard text filter (prefix match)
Filter wildcardFilter = Filter.prefix("job", "doct");

VectorQuery wildcardQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "job", "credit_score", "age", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(wildcardFilter.build())
    .build();

List<Map<String, Object>> wildcardResults = index.query(wildcardQuery);
resultPrint(wildcardResults);

┌────────────┬────────────┬───────────────┬─────────────────┬──────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job   │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼──────┼───────┼───┤
│low         │1741627789  │0              │-122.4194,37.7749│doctor│derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor│nancy  │94 │
└────────────┴────────────┴───────────────┴─────────────────┴──────┴───────┴───┘


In [18]:
// Larger radius - within 100 km
Filter largerGeoFilter = Filter.geo("office_location")
    .radius(-122.4194, 37.7749, 100, Filter.GeoUnit.KM);

VectorQuery largerGeoQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "office_location", "job", "credit_score", "age", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(largerGeoFilter.build())
    .build();

List<Map<String, Object>> largerGeoResults = index.query(largerGeoQuery);
resultPrint(largerGeoResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john   │18 │
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler  │100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO          │taimur │15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist      │joe    │35 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


### Multiple Filter Combinations

You can combine multiple filters using AND and OR logic to create complex query conditions.

In [19]:
// Geographic negation - NOT within 10 km of San Francisco
Filter geoNotFilter = Filter.geo("office_location")
    .notRadius(-122.4194, 37.7749, 10, Filter.GeoUnit.KM);

VectorQuery geoNotQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "office_location", "job", "credit_score", "age", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(geoNotFilter.build())
    .build();

List<Map<String, Object>> geoNotResults = index.query(geoNotQuery);
resultPrint(geoNotResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬──────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user  │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼──────┼───┤
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler │100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim   │12 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO          │taimur│15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist      │joe   │35 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴──────┴───┘


In [20]:
// Larger radius - within 100 km
Filter largergeoFilter = Filter.geo("office_location")
    .radius(-122.4194, 37.7749, 100, Filter.GeoUnit.KM);

VectorQuery largerGeoQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "office_location", "job", "credit_score", "age", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(largergeoFilter.build())
    .build();

List<Map<String, Object>> largerGeoResults = index.query(largerGeoQuery);
resultPrint(largerGeoResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john   │18 │
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler  │100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO          │taimur │15 │
│medium      │1742232589  │0.653301358223 │-122.0839,37.3861│dentist      │joe    │35 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


In [21]:
// Combine multiple filters with AND logic
Filter creditFilter = Filter.tag("credit_score", "high");
Filter ageLowFilter = Filter.numeric("age").gte(18);
Filter ageHighFilter = Filter.numeric("age").lte(100);

// Combine using AND - all conditions must be true
Filter combinedFilter = Filter.and(
    creditFilter,
    ageLowFilter,
    ageHighFilter
);

VectorQuery combinedQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(combinedFilter.build())
    .build();

List<Map<String, Object>> combinedResults = index.query(combinedQuery);
resultPrint(combinedResults);

┌────────────┬────────────┬───────────────┬─────────────────┬────────┬─────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job     │user │age│
├────────────┼────────────┼───────────────┼─────────────────┼────────┼─────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer│john │18 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor  │nancy│94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer│tyler│100│
└────────────┴────────────┴───────────────┴─────────────────┴────────┴─────┴───┘


In [22]:
// Combine multiple filters with AND logic
Filter creditFilter = Filter.tag("credit_score", "high");
Filter ageRangeFilter = Filter.numeric("age").between(18, 100);
Filter jobFilter = Filter.text("job", "engineer");

// Combine using AND - all conditions must be true
Filter combinedFilter = Filter.and(
    creditFilter,
    ageRangeFilter,
    jobFilter
);

VectorQuery combinedQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "credit_score", "age", "job", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(combinedFilter.build())
    .build();

List<Map<String, Object>> combinedResults = index.query(combinedQuery);
resultPrint(combinedResults);

┌────────────┬────────────┬───────────────┬─────────────────┬────────┬─────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job     │user │age│
├────────────┼────────────┼───────────────┼─────────────────┼────────┼─────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer│john │18 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer│tyler│100│
└────────────┴────────────┴───────────────┴─────────────────┴────────┴─────┴───┘


In [23]:
// Combine filters with OR logic
Filter youngFilter = Filter.numeric("age").lt(18);
Filter oldFilter = Filter.numeric("age").gt(93);

// Combine using OR - either condition can be true
Filter orFilter = Filter.or(youngFilter, oldFilter);

VectorQuery orQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "age", "job", "credit_score", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(orFilter.build())
    .build();

List<Map<String, Object>> orResults = index.query(orQuery);
resultPrint(orResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler  │100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO          │taimur │15 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


In [24]:
// Combine filters with OR logic
Filter youngFilter = Filter.numeric("age").between(0, 18);
Filter oldFilter = Filter.numeric("age").between(93, 200);

// Combine using OR - either condition can be true
Filter orFilter = Filter.or(youngFilter, oldFilter);

VectorQuery orQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "age", "job", "credit_score", "office_location", "last_updated", "vector_distance")
    .numResults(10)
    .withPreFilter(orFilter.build())
    .build();

List<Map<String, Object>> orResults = index.query(orQuery);
resultPrint(orResults);

┌────────────┬────────────┬───────────────┬─────────────────┬─────────────┬───────┬───┐
│credit_score│last_updated│vector_distance│office_location  │job          │user   │age│
├────────────┼────────────┼───────────────┼─────────────────┼─────────────┼───────┼───┤
│high        │1741627789  │0              │-122.4194,37.7749│engineer     │john   │18 │
│low         │1741627789  │0              │-122.4194,37.7749│doctor       │derrick│14 │
│high        │1710696589  │0.266666650772 │-122.4194,37.7749│doctor       │nancy  │94 │
│high        │1742232589  │0.109129190445 │-122.0839,37.3861│engineer     │tyler  │100│
│high        │1739644189  │0.158808887005 │-122.0839,37.3861│dermatologist│tim    │12 │
│low         │1742232589  │0.217881977558 │-122.0839,37.3861│CEO          │taimur │15 │
└────────────┴────────────┴───────────────┴─────────────────┴─────────────┴───────┴───┘


## Non-vector Queries

In some cases, you may not want to run a vector query, but just use a filter similar to a SQL query. RedisVL supports pure filter queries.

In [25]:
// Pure filter query without vector similarity
Filter lowCreditFilter = Filter.tag("credit_score", "low");

SearchResult filterResult = index.search(lowCreditFilter.build());

System.out.println("Pure filter query results (low credit): " + filterResult.getTotalResults() + " documents");
filterResult.getDocuments().forEach(doc -> {
    System.out.println("  Document ID: " + doc.getId());
    doc.getProperties().forEach(prop -> {
        if ("user".equals(prop.getKey()) || "credit_score".equals(prop.getKey()) || "job".equals(prop.getKey())) {
            System.out.println("    " + prop.getKey() + ": " + prop.getValue());
        }
    });
    System.out.println();
});

Pure filter query results (low credit): 2 documents
  Document ID: user_queries_docs:derrick
    credit_score: low
    job: doctor
    user: derrick

  Document ID: user_queries_docs:taimur
    credit_score: low
    job: CEO
    user: taimur



In [26]:
// Show query strings
VectorQuery sampleQuery = VectorQuery.builder()
    .vector(new float[]{0.1f, 0.1f, 0.5f})
    .field("user_embedding")
    .returnFields("user", "job")
    .numResults(5)
    .build();

System.out.println("Vector Query String:");
System.out.println("  " + sampleQuery.toQueryString());

Filter tagQuery = Filter.tag("credit_score", "high");
System.out.println("\nTag Filter String:");
System.out.println("  " + tagQuery.build());

// Execute raw query string
SearchResult rawResult = index.search(tagQuery.build());
System.out.println("\nRaw query result count: " + rawResult.getTotalResults());

Vector Query String:
  *=>[KNN $K @user_embedding $vec AS vector_distance]

Tag Filter String:
  @credit_score:{high}

Raw query result count: 4


In [27]:
// Import the CountQuery class
import com.redis.vl.query.CountQuery;

// Example matching Python notebook - count documents with low credit score
Filter hasLowCredit = Filter.tag("credit_score", "low");

CountQuery filterQuery = new CountQuery(hasLowCredit);

long count = index.count(filterQuery);

System.out.println(count + " records match the filter expression " + hasLowCredit.build() + " for the given index.");

2 records match the filter expression @credit_score:{low} for the given index.


## Count Queries

In some cases, you may need to use a filter expression to execute a `CountQuery` that simply returns the count of the number of entities in the pertaining set. The `count()` method provides efficient server-side counting without retrieving the actual documents.

## Cleanup

Clean up the index and close connections:

In [28]:
// Clean up
index.delete(true);
System.out.println("Index deleted");

// Close Redis connection
jedis.close();
System.out.println("Redis connection closed");

Index deleted
Redis connection closed


## Summary

This notebook demonstrated advanced querying capabilities in RedisVL:

1. **Tag Filters**: Exact matches on categorical fields
2. **Numeric Filters**: Range and exact value filters on numeric fields
3. **Text Filters**: Full-text search with exact, wildcard, and fuzzy matching
4. **Geographic Filters**: Radius-based location filtering
5. **Filter Combinations**: AND/OR logic for complex queries
6. **Pure Filter Queries**: Traditional database-style queries without vector similarity
7. **Raw Query Strings**: Direct Redis query execution

These hybrid query capabilities make RedisVL a powerful tool for building sophisticated search applications that combine traditional database operations with vector similarity search.